In [1]:
import os
print(os.getcwd())
os.chdir("../")
print(os.getcwd())
import tensorflow as tf

e:\Preparation\iNeuron\Full_Stack_Data_Science\Deep_Learning\CNN\Project\CNN_Deep_Classifier\research
e:\Preparation\iNeuron\Full_Stack_Data_Science\Deep_Learning\CNN\Project\CNN_Deep_Classifier


In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallBackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_file_path:  Path

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
      root_dir: Path
      trained_model_path: Path
      updated_base_model_path: Path
      training_data_path: Path
      params_epoch : int
      param_batch_size: int
      params_is_augmentation : bool
      params_image_size: list
      


In [4]:
from deepClassifier.constants import *
from deepClassifier.utility import read_yaml_file,create_directories

In [5]:
class ConfigManager:

    def __init__(
        self,
        config_file_path  = CONFIG_FILE_PATH,
        param_file_path  = PARAMS_FILE_PATH
        ):
            self.config = read_yaml_file(config_file_path)
            self.param = read_yaml_file(param_file_path)
            create_directories([self.config.artifact_root])
    
    def get_prepare_callback_config(self) -> PrepareCallBackConfig:
        try:
            config = self.config.prepare_callbacks
            param_config = self.param
            checkpoint_model_dir = os.path.dirname(config.checkpoint_model_file_path)
            create_directories([Path(config.tensorboard_root_log_dir),Path(checkpoint_model_dir)])
            prepare_call_backConfig =   PrepareCallBackConfig(
                                            root_dir = Path(config.root_dir),
                                            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
                                            checkpoint_model_file_path = Path(config.checkpoint_model_file_path)
                                        )
            return prepare_call_backConfig
        except Exception as e:
            raise e

    def get_training_config(self) -> TrainingConfig:
        try:
            config = self.config.training
            create_directories([Path(config.root_dir)])

            trainingConfig = TrainingConfig(
                root_dir = Path(config.root_dir),
                trained_model_path = Path(config.trained_model_path),
                updated_base_model_path = Path(self.config.prepare_base_model.updated_base_model_path),
                training_data_path = Path(os.path.join(self.config.data_ingestion.unzip_dir,"PetImages")),
                params_epoch = self.param.EPOCHS,
                param_batch_size = self.param.BATCH_SIZE,
                params_is_augmentation = self.param.AUGUMENTATION,
                params_image_size = self.param.IMAGE_SIZE
            )

            return trainingConfig
        except Exception as e:
            raise e
    


In [6]:
import time


class PrepareCallBack:

    def __init__(self,config:PrepareCallBackConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timeStamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir,f"tb_logs_at{timeStamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_file_path,
            save_best_only = True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

    

In [7]:
from configparser import Interpolation
from random import shuffle
import time


class Training:

    def __init__(self,config:TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenrator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(

            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.param_batch_size,
            interpolation = "bilinear"
        )

        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenrator_kwargs
        )

        self.valid_generator = valid_data_generator.flow_from_directory(
            directory = self.config.training_data_path,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            training_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2, # slitly tilt image
                zoom_range =  0.2,
                **datagenrator_kwargs
            )
        else:
            training_data_generator = valid_data_generator

        self.train_generator = valid_data_generator.flow_from_directory(
            directory = self.config.training_data_path,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epoch, 
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


    

In [8]:
try:
    config = ConfigManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallBack(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
436/625 [===================>..........] - ETA: 29:49 - loss: 4.8666 - accuracy: 0.7330

e:\Preparation\iNeuron\Full_Stack_Data_Science\Deep_Learning\CNN\Project\CNN_Deep_Classifier\venv\lib\site-packages\PIL\TiffImagePlugin.py:845: UserWarning: Truncated File Read
  warnings.warn(str(msg))


625/625 [==============================] - 7645s 12s/step - loss: 3.9196 - accuracy: 0.7648 - val_loss: 1.6520 - val_accuracy: 0.8141
